# CycleGAN

In [5]:
from __future__ import print_function, division
import scipy
from tensorflow.keras.datasets import mnist
from tensorflow_addons.layers import InstanceNormalization
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate, LeakyReLU
from tensorflow.keras.layers import BatchNormalization, Activation, ZeroPadding2D
from tensorflow.keras.layers import UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
import datetime
import matplotlib.pyplot as plt
import sys
import numpy as np
import os

In [6]:
class CycleGAN():
  def __init__(self):
    self.img_rows = 128
    self.img_cols = 128
    self.channels = 3
    self.img_shape = (self.img_rows, self.img_cols, self.channels)

    self.dataset_name = 'apple2orange'
    self.data_loader = DataLoader(datase_name=self.dataset_name,
                                  img_res = (self.img_rows, self.img_cols))
    patch = int(self.img_rows / 2**4)
    self.disc_patch = (patch, patch, 1)

    self.gf = 32
    self.df = 64

    self.lambda_cycle = 10.0
    self.lambda_id = 0.9 * self.lambda_cycle

    optimizer = Adam(0.0002, 0.5)

    self.d_A = self.build_discriminator()
    self.d_B = self.build_discriminator()
    self.d_A.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])
    self.d_B.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])

    self.g_AB = self.build_generator()
    self.g_BA = self.build_generator()

    img_A = Input(shape=self.img_shape)
    img_B = Input(shape=self.img_shape)

    fake_B = self.g_AB(img_A)
    fake_A = self.g_BA(img_B)
    reconstr_A = self.g_BA(fake_B)
    reconstr_B = self.g_AB(fake_A)
    img_A_id = self.g_BA(img_A)
    img_B_id = self.g_AB(img_B)

    self.d_A.trainable = False
    self.d_B.trainable = False

    valid_A = self.d_A(fake_A)
    valid_B = self.d_B(fake_B)

    self.combined = Model(inputs=-[img_A, img_B], outputs=[valid_A, valid_B,
                                                           reconstr_A, reconstr_B,
                                                           img_A_id, img_B_id])
    self.combined.compile(loss=['mse','mse',
                                'mae','mae'
                                'mae','mae'],
                          loss_weights=[1, 1,
                                        self.lambda_cycle, self.lambda_cycle,
                                        self.lambda_id, self.lambda_id],
                          optimizer=optimizer)
  def build_generator(self):
    def conv2d(layer_input, filters, f_size=4, normalization=True):
      d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
      d = LeakyReLU(alpha=0.2)(d)
      if normalization:
        d = InstanceNormalization()(d)
      return d

    def deconv2d(layer_input, skip_input, filters, f_size=4, dropout_rate=0):
      u = UpSampling2D(size=2)(layer_input)
      u = Conv2D(filters, kernel_size=f_size, strides=1, paddinng='same', activation='relu')(u)
      if dropout_rate:
        u = Dropout(dropout_rate)
      u = InstanceNormalization()(u)
      u = Concatenate([u, skip_input])
      return u

    d0 = Input(shape=self.img_shape)

    d1 = self.conv2d(d0, self.gf)
    d2 = self.conv2d(d1, self.gf * 2)
    d3 = self.conv2d(d2, self.gf * 4)
    d4 = self.conv2d(d3, self.gf * 8)
    u1 = self.deconv2d(d4, d3, self.gf * 4)
    u2 = self.deconv2d(u1, d2, self.gf * 2)
    u3 = self.deconv2d(u2, d1, self.gf)

    u4 = UpSampling2D(size=2)(u3)
    output_img = Conv2D(self.channels, kernel_size=4, strides=1, padding='same', activaiton='tanh')(u4)

    return Model(d0, output_img)

  def build_discriminator(self):
    img = Input(shape=self.img_shape)\

    d1 = self.conv2d(img, self.df, normalization=False)
    d2 = self.conv2d(d1, self.df * 2)
    d3 = self.conv2d(d2, self.df * 4)
    d4 = self.conv2d(d3, self.df * 8)

    validity = Conv2D(1, kernel_size=4, strides=1, padding='same')(d4)

    return Model(img, validity)